In [1]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
# this model have accuracy of 100% and loss of 0.0015
model = load_model('latest3_rgb.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle','middle']

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Load your pre-trained CNN model
def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes


# Define the desired dimensions
desired_width = 250
desired_height = 100


# Initialize the webcam
video_capture = cv2.VideoCapture(0)  # 0 represents the default webcam device
eyes_detected = False

x = 0

list1=[]

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    eyes = detect_eyes(frame)
    list1.append(eyes)

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]

        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        # eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        # eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)

        if eye_x == eye_1x:
            left_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
            right_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
        else:
            left_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
            right_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
        
        # Resize the ROI to match the desired dimensions
        resized_left_eye_roi = cv2.resize(left_eye_roi, (125, 100))
        resized_right_eye_roi = cv2.resize(right_eye_roi, (125, 100))
        
        combined_image = cv2.hconcat([resized_left_eye_roi, resized_right_eye_roi])



        # cv2.imwrite(f'test/{x}.jpg', combined_image)
        # x +=1
        
        # eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        # resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to RGB format
        resized_roi = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)

        # Convert the ROI to numpy array
        resized_roi = np.asarray(resized_roi)

        # Expand dimensions to match the input shape of your CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi / 255.0

        # Pass the preprocessed ROI through your CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)
        # print(predicted_class)
        # frame = cv2.rectangle(frame, (eye_x, eye_y), (eye_x+eye_width, eye_y+eye_height), (0, 255, 0), 2)
        for (x,y,w,h) in eyes:
            frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
    #     eyes_detected = True
    # else:
    #     eyes_detected = False
        
    # frame = cv2.flip(frame, 1)

    # if eyes_detected:
    cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 41ms/step


In [13]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import random
import pyautogui







# Load the pre-trained CNN model
model = load_model('latest3_rgb.h5')
folders = ['bottom_left', 'bottom_right', 'bottom_middle', 'top_left', 'top_right', 'top_middle', 'left_middle', 'right_middle', 'middle']
folders_number = [6, 8, 7, 0, 2, 1, 3, 5, 4]

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

def detect_eyes(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(30, 30))
    return eyes

# Define the desired dimensions
desired_width = 250
desired_height = 100

# Create the black window
black_window = np.zeros((700, 1400, 3), dtype=np.uint8)
# rectangle_center = [[103, 53], [633, 53], [1163, 53], [103, 343], [633, 343], [1163, 343], [103, 633], [633, 633], [1163, 633]]
rectangle_center = []

x_cor= [100, 1000, 1900]
y_cor = [100, 550, 1000]

for i in range(3):
    for j in range(3):
        rectangle_center.append([x_cor[j], y_cor[i]])


green_rectangle_coordinates = []
for i in range(2):
    for j in range(2):
        green_rectangle_coordinates.append([3+j*530, 3+i*290])
        # rectangle_center.append([3+j*530+100, 3+i*290+50])

print(rectangle_center)
# Initialize the webcam
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame from the webcam
    ret, frame = video_capture.read()
    frame = cv2.flip(frame, 1)
    # cv2.imshow('Game', black_window)

    eyes = detect_eyes(frame)
    

    if len(eyes) >= 2:
        (eye_1x, eye_1y, eye_1width, eye_1height), (eye_2x, eye_2y, eye_2width, eye_2height) = eyes[:2]
        eye_x = min(eye_1x, eye_2x)
        eye_y = min(eye_1y, eye_2y)
        # eye_width = abs(eye_1x - eye_2x) + max(eye_1width, eye_2width)
        # eye_height = abs(eye_1y - eye_2y) + max(eye_1height, eye_2height)
        
        if eye_x == eye_1x:
            left_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
            right_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
        else:
            left_eye_roi = frame[eye_2y:eye_2y + eye_2height, eye_2x:eye_2x + eye_2width]
            right_eye_roi = frame[eye_1y:eye_1y + eye_1height, eye_1x:eye_1x + eye_1width]
        
        # Resize the ROI to match the desired dimensions
        resized_left_eye_roi = cv2.resize(left_eye_roi, (125, 100))
        resized_right_eye_roi = cv2.resize(right_eye_roi, (125, 100))
        
        combined_image = cv2.hconcat([resized_left_eye_roi, resized_right_eye_roi])

        # eye_roi = frame[eye_y:eye_y+eye_height, eye_x:eye_x+eye_width]

        # Resize the ROI to match the desired dimensions
        # resized_roi = cv2.resize(eye_roi, (desired_width, desired_height))

        # Convert the ROI to grayscale
        resized_roi = cv2.cvtColor(combined_image, cv2.COLOR_BGR2RGB)

        # Expand dimensions to match the input shape of the CNN model
        resized_roi = np.expand_dims(resized_roi, axis=0)

        # Normalize pixel values
        resized_roi = resized_roi.astype('float32') / 255

        # Pass the preprocessed ROI through the CNN model for prediction
        predictions = model.predict(resized_roi)
        predicted_class = np.argmax(predictions, axis=1)

        # Get the real class number
        real_class = folders_number[predicted_class[0]]
        
        selected_coordinates = rectangle_center[real_class]

        # Move the mouse cursor to the specified coordinates
        pyautogui.moveTo(selected_coordinates[0], selected_coordinates[1], duration=0.5)
        # Remove the selected coordinates from the list
        # for i in green_rectangle_coordinates:
        #     if abs(selected_coordinates[0]- i[0]) <= 100 and abs(selected_coordinates[1]- i[1]) <= 50:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
        #         break
            
        #     elif abs(selected_coordinates[0]- i[0]) <= 125 and abs(selected_coordinates[1]- i[1]) <= 63:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
        #         break
                
            
        #     elif abs(selected_coordinates[0]- i[0]) <= 150 and abs(selected_coordinates[1]- i[1]) <= 75:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
        #         break
                
            
        #     elif abs(selected_coordinates[0]- i[0]) <= 175 and abs(selected_coordinates[1]- i[1]) <= 83:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
        #         break
                
                
        #     elif abs(selected_coordinates[0]- i[0]) <= 200 and abs(selected_coordinates[1]- i[1]) <= 100:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                
        #         break
            
        #     elif abs(selected_coordinates[0]- i[0]) <=250  and abs(selected_coordinates[1]- i[1]) <= 125:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
                
        #         break
            
        #     elif abs(selected_coordinates[0]- i[0]) <=300  and abs(selected_coordinates[1]- i[1]) <= 150:
        #         green_rectangle_coordinates.remove(i)
        #         green_rectangle_coordinates.append([random.randint(0, 1200), random.randint(0, 600)])
        #         break



        for (x,y,w,h) in eyes:
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 255, 0), 2)
        frame = cv2.putText(frame, folders[predicted_class[0]], (eye_x, eye_y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    # if len(green_rectangle_coordinates) < 9:
    #     green_rectangle_coordinates.append([random.randint(0, 1593), random.randint(0, 873)])
        

    
    # Clear the black window
    # black_window.fill(0)

    # for [x, y] in green_rectangle_coordinates:
    #     cv2.rectangle(black_window, (x, y), (x+200, y+100), (0, 255, 0), cv2.FILLED)
        
    

    cv2.imshow('Eye Detection', frame)
    

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close the windows
video_capture.release()
cv2.destroyAllWindows()


[[100, 100], [1000, 100], [1900, 100], [100, 550], [1000, 550], [1900, 550], [100, 1000], [1000, 1000], [1900, 1000]]
1/1 [==============================] - 0s 32ms/step
